In [18]:
import psycopg2
from psycopg2 import sql


In [19]:

# Bağlantı parametreleri (Docker container)
DB_HOST = "localhost"       # Docker host
DB_PORT = "5432"            # Docker port
DB_USER = "postgres"        # PostgreSQL kullanıcı adı
DB_PASSWORD = "210705"      # PostgreSQL şifresi


In [20]:


conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    dbname="postgres"
)
conn.autocommit = True
cur = conn.cursor()


db_name = "new_db"
cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
print(f"veritabanı oluşturuldu: {db_name}")

cur.close()
conn.close()

conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    dbname=db_name
)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100)
)
""")
conn.commit()
print("✅ Tablo 'users' oluşturuldu")


veritabanı oluşturuldu: new_db
✅ Tablo 'users' oluşturuldu


### veri ekleme

In [14]:
cur = conn.cursor()#önceki bağlantıyı kullanıyoruz

In [21]:
users_to_insert = [
    ("ahsen", "ahsenbulbul21@gmail.com"),
    ("dilara", "dido@mido.com"),
    ("müzeyyen", "muz@ornek.com")
]

In [22]:
insert_query = "INSERT INTO users (name, email) VALUES (%s, %s)"#sql sorgumuz 
cur.executemany(insert_query, users_to_insert)#burada ilk parametre sorgu ikincisi ise eklemek istediğimiz veriler
conn.commit()#değişiklikleri kaydediyoruz
print("✅ Kullanıcılar tabloya eklendi")

✅ Kullanıcılar tabloya eklendi


### veri çekme

In [23]:
cur.execute("SELECT id, name, email FROM users")
rows = cur.fetchall()
print("📊 Kullanıcılar tablodan çekildi:")
for row in rows:
    print(row)

📊 Kullanıcılar tablodan çekildi:
(1, 'ahsen', 'ahsenbulbul21@gmail.com')
(2, 'dilara', 'dido@mido.com')
(3, 'müzeyyen', 'muz@ornek.com')


### güncelleme

In [24]:
cur.execute("UPDATE users SET email = %s WHERE name = %s", ("ahsen_new@example.com", "ahsen"))#mailimi degistiriom
conn.commit()
print("✅ Kullanıcı güncellendi")

✅ Kullanıcı güncellendi


In [25]:
cur.execute("SELECT id, name, email FROM users")
rows = cur.fetchall()
print("📊 Güncellenmiş kullanıcılar:")
for row in rows:
    print(row)

📊 Güncellenmiş kullanıcılar:
(2, 'dilara', 'dido@mido.com')
(3, 'müzeyyen', 'muz@ornek.com')
(1, 'ahsen', 'ahsen_new@example.com')


### silme

In [ ]:
cur.execute("DELETE FROM users WHERE name = %s", ("müzeyyen",))
conn.commit()
print("✅ Kullanıcı silindi")